# OGC API Processes demonstratie client

Dit is het notebook wat hoort bij de gecombineerde Pygeoapi / jupyternotebook in docker configuratie.

Omdat je nu een verbinding moet leggen tussen de twee docker containers zijn de url's van pygeoapi net iets anders: http://pygeoapi

Met de eerste regel hieronder voer je de commando's uit om de benodigde python packages te installeren.


In [ ]:
!pip install folium
!pip install requests
!pip install geopandas

In [ ]:
import folium
import requests  # For making HTTP requests
import geopandas as gpd
import json

In [ ]:
url = "http://pygeoapi/processes/"  
 
response = requests.get(url)

response.text

In [ ]:
# details van een specifiek proces

url = "http://pygeoapi/processes/geom-details"

headers = {"Content-Type": "application/json"}

response = requests.get(url, headers=headers)
response.json()

In [ ]:
# uitvoeren van een proces

url = "http://pygeoapiprocesses/geom-details/execution"  
data = {"inputs": {"polyWKT": "POLYGON ((73.65865031385647 17.77623164372008, 73.65897114729012 17.777001795974, 73.65825595609473 "
"17.776963606683765, 73.65817351972652 17.776647483916918, 73.65852108927962 17.7764247125032, 73.65865031385647 17.77623164372008))"}}

headers = {"Content-Type": "application/json"}

response = requests.post(url, json=data, headers=headers)

response.json()

In [ ]:
def handle_response(response):
  if response.status_code == 200:
    
    return response.json()
    
  else:
    print(f"Error: {response.status_code, response.json()['description']}")

In [ ]:
url = "http://pygeoapi/processes/"  

headers = {"Content-Type": "application/json"}

response = requests.get(url, headers=headers)
processes =  handle_response(response)

processes

Kaart met de waarneemstations (OGC API Features)

In [ ]:
m = folium.Map(location=[52.2, 5.5], zoom_start=8)

url = "http://pygeoapi/collections/knmi_meetstations/items?f=json&limit=1000"

# Fetch the GeoJSON data
gdf = gpd.read_file(url)

#quick hack omdat de datetime kolom niet goed naar geojson geconverteerd wordt...
gdf_subset = gdf[['STN','TYPE', 'geometry']]

# Convert to GeoJSON
geojson = gdf_subset.to_json()

folium.GeoJson(
    gdf_subset,
    name="Stations",
    style_function=lambda x: {'fillColor': 'blue', 'color': 'black', 'weight': 1, 'fillOpacity': 0.5},
    tooltip=folium.GeoJsonTooltip(fields=["STN"]),
    popup=folium.GeoJsonPopup(fields=["STN", "TYPE"]),
    highlight_function=lambda x: {"fillOpacity": 0.8},
).add_to(m)

m

Kaart met het resultaat van de classificatie door het OGC API Processes localoutlier algoritme 

In [ ]:
def calc_lof(feat):
    url = "http://pygeoapi/processes/localoutlier/execution"  
    data = {"inputs": {"dataset": feat}}
    headers = {"Content-Type": "application/json"}

    response = requests.post(url, json=data, headers=headers)
    return handle_response(response)


#let op: omdat dit de url is vanaf de ogc api process functie naar de ogc api features op dezelfde server is de url zonder poort 5000
input = 'http://localhost/collections/knmi_meetstations/items?f=json&limit=1000' 
result = calc_lof(input)

print(result)
with open('meetstations.geojson', 'w') as f:
    json.dump(json.loads(result['value']), f)

m = folium.Map(location=[52.2, 5.5], zoom_start=8)
gdf = gpd.read_file('meetstations.geojson')
gdf2 = gdf[['STN','TYPE', 'geometry','abnormality']]


folium.GeoJson(
    gdf2,
    name="Stations",
    marker=folium.Marker(icon=folium.Icon(icon='star')),
    tooltip=folium.GeoJsonTooltip(fields=["STN","abnormality"]),
    popup=folium.GeoJsonPopup(fields=["STN", "TYPE"]),
    style_function=lambda x: {
        'markerColor': 'orange' if x['properties']['abnormality'] == -1 else 'green',
        'color': 'black',
        'weight': 2,
        'fillOpacity': 0.7
    },
    highlight_function=lambda x: {"fillOpacity": 0.8},
).add_to(m)

value_counts = gdf['abnormality'].value_counts()
print(value_counts)
m